In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import fileinput

In [33]:
quartiles = 'q1|q2|q3' #arg
quartiles = quartiles.lower()
drop_without_quartile = False if 'q4' in quartiles else True

In [34]:
core_df = pd.read_excel('scopus-ITMO.xlsx', na_values='Q-нет'). \
loc[:, ['Авторы', 'Квартиль', 'Идентификаторы авторов']]
core_df = core_df.loc[core_df.loc[:, 'Авторы'].notnull(), :]
print(len(core_df))
core_df = core_df.loc[core_df.loc[:, 'Квартиль'].str. \
                      lower().str. \
                      contains(quartiles,na=(not drop_without_quartile)) , :]
core_df = core_df.loc[:, ['Авторы', 'Идентификаторы авторов']].rename(columns={'Идентификаторы авторов':'ids'})
print(len(core_df))
core_df.head()

9223
4752


,Авторы,ids
0,"Alowayyed S., Piontek T., Suter J.L., Hoenen O...",56610399200;56635198100;16744711000;2233432910...
8,"Alfimov G.L., Gegel L.A., Lebedev M.E., Malome...",6603433683;57202706334;56735800900;57194699446...
9,"Zhuk N.A., Chezhina N.V., Belyy V.A., Makeev B...",15763942200;6701522471;57190393910;26664800600...
10,"Cha J.H., Finkelstein M., Levitin G.",7202455739;57195280575;7006743313;
11,"Perovskiy I.A., Khramenkova E.V., Pidko E.A., ...",57203415672;56460365300;12778911000;1277852880...


has_scopus_name = lambda row: False if row['Издание'].find('Scopus') == -1 else True
core_df['Scopus'] = core_df.apply(has_scopus_name, axis=1)
core_df = core_df[core_df['Scopus']]

row = core_df.loc[a, :].iloc[0]
row

In [35]:
def split(stucked_str, delimiter):
    srs = pd.Series(stucked_str.split(delimiter))
    srs = srs.str.strip()
    return srs

names = split((row['Авторы'] + ', JR., kjjk.f'), ',')
names[~names.str.contains('JR.')]

authors = {}
authors_pairs = {}

names = split(row['Авторы'], ',')
if (any(names.str.contains('JR', case=False) == True)): #what does 'JR.' after name means? skip those
    print("FUUUUuuUUUUUUUUU")
ids = split(row['ids'], ';')
ids = ids[ids.str.isnumeric()] # filter empty strings

names = split(row['Авторы'], ',')
if (any(names.str.contains('JR', case=False) == True)): #what does 'JR.' after name means? skip those
    print("FUUUUuuUUUUUUUUU")
ids = split(row['ids'], ';')
ids = ids[ids.str.isnumeric()] # filter empty strings

for i in range(len(names)):
    authors.setdefault(ids[i], {'name':names[i], 'pub':0}) # add to dict 'authors' [id] -> (name, 0 {pulications})
    authors[ids[i]]['pub'] += 1

ids_pairs = combinations(ids, 2)
for pair in ids_pairs:
    pair = tuple(sorted(pair))
    if pair[0] == pair[1]:
        continue
    authors_pairs.setdefault(pair, 0)
    authors_pairs[pair] += 1

In [36]:
def process_row(row):
    names = split(row['Авторы'], ',')
    if (any((names.str.len() <= 3) == True)):   #what does names with less than 3 characters
                                                #mean? skip those
        return
    ids = split(row['ids'], ';')
    ids = ids[ids.str.isnumeric()] # filter empty strings
#     if len(names) > len(ids):
#         return True
#         names = names[:len(ids)]
#     if len(names) < len(ids):
#         return True
#         ids = ids[:len(names)]
    
    for i in range(len(names)):
        authors.setdefault(ids[i], {'name':names[i], 'count':0}) # add to dict 'authors' [id] -> (name, 0 {pulications})
        authors[ids[i]]['count'] += 1

    ids_pairs = combinations(ids, 2)
    for pair in ids_pairs:
        pair = tuple(sorted(pair))
        if pair[0] == pair[1]:
            continue
        authors_pairs.setdefault(pair, 0)
        authors_pairs[pair] += 1
    return

In [37]:
authors = {}
authors_pairs = {}

_ = core_df.apply(func=process_row, axis=1)

authors = pd.DataFrame.from_dict(authors, orient='index')
authors = authors.reset_index().rename(columns={'index':'id'}).set_index('id')
authors_pairs = pd.DataFrame.from_dict(authors_pairs, orient='index')

authors_pairs = authors_pairs.reset_index().rename(columns={0:'count'})

authors_pairs[['id1', 'id2']] = authors_pairs.loc[:, 'index'].apply(lambda pair: pd.Series([pair[0], pair[1]]))

authors_pairs = authors_pairs.loc[:, ['count', 'id1', 'id2']]

print(authors.head())
print(authors_pairs.head())

                       name  count
id                                
10038963900    Merks R.M.H.      1
10039857500       Clarke E.      4
10039922200    Leushin V.V.      3
10040023500  Astakhov Yu.S.      1
10041592700    Karasev P.A.      3
   count          id1          id2
0      1  56610399200  56635198100
1      1  16744711000  56610399200
2      1  22334329100  56610399200
3      2  23049787700  56610399200
4      1  56610399200  57203961704


author_count = core_df.loc[:, ['AuthorID', 'Название публикации']]. \
groupby(by=['AuthorID']).count(). \
rename(columns={'Название публикации':'Публикаций'})
print(len(author_count))
author_count.head()

author_count = author_count.merge(core_df.loc[:, ['Авторы', 'AuthorID']].drop_duplicates('AuthorID'),
                   left_index=True, right_on='AuthorID',
                   validate='one_to_one')
print(len(author_count))
author_count.head()

group_pubs = core_df.loc[:, ['AuthorID', 'Авторы', 'Название публикации']]. \
groupby(by=['Название публикации'])

pairs_pub = {}
def process_group_of_authors(df):
    ids = df.loc[:, 'AuthorID']
    ids_pairs = combinations(ids, 2)
    for pair in ids_pairs:
        pair = tuple(sorted(pair))
        if pair[0] == pair[1]:
            continue
        pairs_pub.setdefault(pair, 0)
        pairs_pub[pair] += 1
        
group_pubs.apply(process_group_of_authors)

pairs_count = pd.DataFrame.from_dict(pairs_pub, orient='index', columns=['count'])
pairs_count = pairs_count.reset_index()
pairs_count[['id1', 'id2']] = pairs_count.loc[:, 'index'].apply(pd.Series)
pairs_count.drop(columns=['index'], inplace=True)
pairs_count.head()

In [38]:
authors.head()

,name,count
id,,
10038963900,Merks R.M.H.,1
10039857500,Clarke E.,4
10039922200,Leushin V.V.,3
10040023500,Astakhov Yu.S.,1
10041592700,Karasev P.A.,3


In [39]:
authors = authors.reset_index()

In [40]:
#convert dataset to vis.js format
edges = authors_pairs.rename(columns={'count':'value', 'id1':'from', 'id2':'to'})
edges.loc[:, 'title'] = 'Совместных работ: ' + edges.loc[:, 'value'].astype(str)

nodes = authors.rename(columns={'name':'label', 'count':'mass'}) #title
nodes.loc[:, 'value'] = np.sqrt(nodes.loc[:, 'mass'] / np.pi)
nodes.loc[:, 'title'] = 'Публикаций: ' + nodes.loc[:, 'mass'].astype(str)
nodes.loc[:, 'group'] = np.random.randint(1, 9, nodes.shape[0])
print(nodes.head())
print(edges.head())

            id           label  mass     value          title  group
0  10038963900    Merks R.M.H.     1  0.564190  Публикаций: 1      1
1  10039857500       Clarke E.     4  1.128379  Публикаций: 4      2
2  10039922200    Leushin V.V.     3  0.977205  Публикаций: 3      3
3  10040023500  Astakhov Yu.S.     1  0.564190  Публикаций: 1      1
4  10041592700    Karasev P.A.     3  0.977205  Публикаций: 3      6
   value         from           to                title
0      1  56610399200  56635198100  Совместных работ: 1
1      1  16744711000  56610399200  Совместных работ: 1
2      1  22334329100  56610399200  Совместных работ: 1
3      2  23049787700  56610399200  Совместных работ: 2
4      1  56610399200  57203961704  Совместных работ: 1


In [38]:
# required for faster physics calculation
edges.loc[:, 'hidden'] = True
print(edges.head())

   value         from           to                title  hidden
0      1  56610399200  56635198100  Совместных работ: 1    True
1      1  16744711000  56610399200  Совместных работ: 1    True
2      1  22334329100  56610399200  Совместных работ: 1    True
3      2  23049787700  56610399200  Совместных работ: 2    True
4      1  56610399200  57203961704  Совместных работ: 1    True


In [41]:
def replace_str(filename, text_to_search, replacement_text):
    with fileinput.FileInput(filename, inplace=True) as file:
        for line in file:
            print(line.replace(text_to_search, replacement_text), end='')

In [42]:
edges_filename = 'edges_ITMO ' + quartiles + '.json'
nodes_filename = 'nodes_ITMO ' + quartiles + '.json'

### Запись в файл (без координат)

In [12]:
# Вывод в файл

edges.to_json(edges_filename, orient='records', force_ascii=True)
nodes.to_json(nodes_filename, orient='records', force_ascii=True)

replace_str(nodes_filename, '"title"', ' title ')
replace_str(nodes_filename, '"value"', ' value ')
replace_str(nodes_filename, '"group"', ' group ')
replace_str(nodes_filename, '"id"', ' id ')
replace_str(nodes_filename, '"label"', ' label ')
replace_str(nodes_filename, '"mass"', ' mass ')
replace_str(nodes_filename, '"x"', ' x ')
replace_str(nodes_filename, '"y"', ' y ')
# replace_str('pers_pub_net.json', '"physics"', ' physics ')
# replace_str('pers_pub_net.json', '"false"', ' false ')

replace_str(edges_filename, '"value"', ' value ')
replace_str(edges_filename, '"from"', ' from ')
replace_str(edges_filename, '"title"', ' title ')
replace_str(edges_filename, '"to"', ' to ')
replace_str(edges_filename, '"hidden"', ' hidden ')

## setting up positions of nodes

In [43]:
positions = pd.read_json('ITMO_positions_' + quartiles + '.json', orient='records')
print(len(positions))
positions.head()

9409


,id,x,y
0,10038963900,-36094,-45441
1,10039857500,-7437,4515
2,10039922200,46700,-48050
3,10040023500,23791,12174
4,10041592700,-754,-21443


In [44]:
positions.loc[:, 'id'] = positions.loc[:, 'id'].astype(int)
nodes.loc[:, 'id'] = nodes.loc[:, 'id'].astype(int)

In [45]:
nodes_with_coord = nodes.merge(right=positions, left_on='id', right_on='id', validate='one_to_one')
nodes_with_coord = nodes_with_coord.sort_values('mass', ascending=False)

In [46]:
nodes_with_coord.head()

,id,label,mass,value,title,group,x,y
8402,7006167582,Belov P.,141,6.699380,Публикаций: 141,8,-5905,-14031
1352,36038963000,Kivshar Y.,131,6.457445,Публикаций: 131,2,-1995,-11735
8824,7201565128,Baranov A.,120,6.180387,Публикаций: 120,7,-426,-28172
142,14008386700,Fedorov A.V.,109,5.890312,Публикаций: 109,8,4145,-26842
7768,7003772604,Nikonorov N.V.,101,5.670035,Публикаций: 101,4,-12641,-16617


In [47]:
edges.to_json(edges_filename, orient='records', force_ascii=True)
nodes_with_coord.to_json(nodes_filename, orient='records', force_ascii=True)

replace_str(nodes_filename, '"title"', ' title ')
replace_str(nodes_filename, '"value"', ' value ')
replace_str(nodes_filename, '"group"', ' group ')
replace_str(nodes_filename, '"id"', ' id ')
replace_str(nodes_filename, '"label"', ' label ')
replace_str(nodes_filename, '"mass"', ' mass ')
replace_str(nodes_filename, '"x"', ' x ')
replace_str(nodes_filename, '"y"', ' y ')
# replace_str('pers_pub_net.json', '"physics"', ' physics ')
# replace_str('pers_pub_net.json', '"false"', ' false ')

replace_str(edges_filename, '"value"', ' value ')
replace_str(edges_filename, '"from"', ' from ')
replace_str(edges_filename, '"title"', ' title ')
replace_str(edges_filename, '"to"', ' to ')
replace_str(edges_filename, '"hidden"', ' hidden ')

stucked_names = core_df.loc[:, 'Авторы']
stucked_names.head()

def split_names(stucked_names_entry):
    splitted = stucked_names_entry.split('\n')
    str_list = []
    for substr in splitted:
        temp = ' '.join(substr.split())
        str_list.append(temp)
    str_list = list(filter(None, str_list))
    return str_list

def pairwise(t):
    it = iter(t)
    return zip(it,it)

def detect_faculty(str_with_faculty):
    selector = {
        str_with_faculty.find('МТ'): 'МТ',
        str_with_faculty.find('ИУ'): 'ИУ',
        str_with_faculty.find('РЛ'): 'РЛ',
        str_with_faculty.find('ФН'): 'ФН',
        str_with_faculty.find('СМ'): 'СМ',
        str_with_faculty.find('Э'): 'Э',
        str_with_faculty.find('РК'): 'РК',
        str_with_faculty.find('БМТ'): 'БМТ',
        str_with_faculty.find('Л'): 'Л',
        str_with_faculty.find('ИБМ'): 'ИБМ',
        str_with_faculty.find('СГН'): 'СГН',
        str_with_faculty.find('РКТ'): 'РКТ',
        str_with_faculty.find('АК'): 'АК',
        str_with_faculty.find('ПС'): 'ПС',
        str_with_faculty.find('РТ'): 'РТ',
        str_with_faculty.find('ОЭП'): 'ОЭП',
        str_with_faculty.find('ЮР'): 'ЮР',
        str_with_faculty.find('ГУИМЦ'): 'ГУИМЦ',
        str_with_faculty.find('ФМОП'): 'ФМОП',
        str_with_faculty.find('ФВО'): 'ФВО',
    }
    try:
        sel = selector[True]
        assert(type(sel) == str), 'Multiple faculties detected: {}'.format(type(sel))
        return sel
    except KeyError as e:
        return 'Other'

def process_group_of_names_df(group_of_names, personal_publications_df, pair_publications_df):
    splitted = split_names(group_of_names)
    name_position_pairs = pairwise(splitted)
    bmstu_members = [entry for entry in name_position_pairs if (entry[1].find('Не МГТУ') == -1)]
    
    
    
def process_group_of_names(group_of_names):
    splitted = split_names(group_of_names)
    name_position_pairs = pairwise(splitted)
    bmstu_members = [entry for entry in name_position_pairs if (entry[1].find('Не МГТУ') == -1)]
    
    for entry in bmstu_members:
        faculty = detect_faculty(entry[1])
        personal_publications_num.setdefault(entry[0], [0, faculty])
        personal_publications_num[entry[0]][0] += 1
    for single_comb in combinations(bmstu_members, 2):
        sorted_comb = sorted(single_comb, key=lambda tup: tup[0])
        names_comb = (sorted_comb[0][0], sorted_comb[1][0])
        pair_publications_num.setdefault(names_comb, 0)
        pair_publications_num[names_comb] += 1

personal_publications_num = {}
pair_publications_num = {}
stucked_names = core_df['Авторы']
stucked_names.apply(process_group_of_names);

In [ ]:
personal_publications_num_df = pd.DataFrame.from_dict(personal_publications_num, orient='index',
                                                      columns=['pub_num', 'faculty'])
personal_publications_num_df.index.names = ['full_name']
personal_publications_num_df.reset_index(inplace=True)
personal_publications_num_df.loc[:, 'id'] = personal_publications_num_df.apply(lambda row: id(row['full_name']),
                                                                               axis=1)


pair_publications_num_df = pd.DataFrame.from_dict(pair_publications_num, orient='index', columns=['pair_pub_num'])
pair_publications_num_df.index.names = ['names_pair']
pair_publications_num_df.reset_index(inplace=True)
get_ids = lambda row: pd.Series([id(row['names_pair'][0]),
                                 id(row['names_pair'][1])])
pair_publications_num_df[['name1_id', 'name2_id']] = pair_publications_num_df.apply(get_ids, axis=1)

def append_key_data(row):
     return row.loc['full_name'] + ' (' + str(row.loc['pub_num']) + ', ' + row.loc['faculty'] + ')'

personal_publications_num_df['key_data'] = personal_publications_num_df.apply(
    append_key_data, axis=1)

personal_publications_num_network_format_df = personal_publications_num_df.rename(
    columns={'key_data':'title', 'pub_num':'value', 'faculty':'group'})[['title', 'value', 'group', 'id']]
pair_publications_num_network_format_df = pair_publications_num_df.rename(
    columns={'pair_pub_num':'value', 'name1_id':'from', 'name2_id':'to'})[['value', 'from', 'to']]

personal_publications_num_network_format_df.to_json('pers_pub_net.json', orient='records')
pair_publications_num_network_format_df.to_json('pair_pub_net.json', orient='records')